In [4]:
import csv
import jieba
import pickle
import gensim
import numpy as np
from sklearn import metrics

In [10]:
w2v_model = gensim.models.Word2Vec.load('w2v_model_size100_window5_mincount1')

class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(list(word2vec.values())[0])

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    
vectorizer = MeanEmbeddingVectorizer(dict(zip(w2v_model.wv.index2word, w2v_model.wv.syn0)))
w2v_svrs = pickle.load(open('./pickles/w2v_svrs.p','rb'))

dummy = jieba.cut('',cut_all = False)

def jieba_cut(sentence):
    return [word for word in jieba.cut(sentence, cut_all = False) if word != ' ']

def w2v_svr_score(cut):
    clist = list(cut)
    filtered_list = []
    for w in clist:
        try:
            s = w2v_model.wv[w]
            filtered_list.append(w)
        except KeyError:
            continue
    return np.mean([svr.predict(X=list(vectorizer.transform([list(cut)]))) for svr in w2v_svrs], axis=0)

In [11]:
randomForest_clf = pickle.load(open('./pickles/TFIDF_RandomForestClassifier.pkl','rb'))
linearSVC_clf = pickle.load(open('./pickles/LinearSVC_Classifier.pkl','rb'))
naive_bayes_clf = pickle.load(open('./pickles/Naive_bayes_Classifier.pkl','rb'))
decisionTree_clf = pickle.load(open('./pickles/DecisionTree_Classifier.pkl', 'rb'))

In [12]:
data_folder = './emotiondict/ANTUSD/opinion_word_utf8.csv'

word_list = []
score_list = []

f = open(data_folder, 'r')  
for row in csv.reader(f):  
    word_list.append(row[0])
    score_list.append(row[1])
    

pos_neutral_therehold = 0.05
neg_neutral_therehold = -0.05
pos_therethold = 0.2
neg_therethold = -0.2
def emotion_score_calculator(content):
    emotion_score = 0
    number = 0
#     print(content)
    for word in content:
        if word in word_list:
            index = word_list.index(word)
#             print(index, word, float(score_list[index]))
            emotion_score += float(score_list[index])
            number += 1

    if number == 0:
        return emotion_score
    else:
        normalize_score = emotion_score / number
        emotion_degree = ['非常滿意', '滿意', '中立', '生氣', '非常生氣']
        if normalize_score > 0:
            if normalize_score < pos_neutral_therehold:
                return (normalize_score , emotion_degree[2])
            elif normalize_score < pos_therethold:
                return (normalize_score , emotion_degree[1])
            else:
                return (normalize_score , emotion_degree[0])
        else:
            if normalize_score > neg_neutral_therehold:
                return (normalize_score , emotion_degree[2])
            elif normalize_score > neg_therethold:
                return (normalize_score , emotion_degree[3])
            else:
                return (normalize_score , emotion_degree[4])

# randomForest_clf.predict(['那一個 小時 預約 快 修   的 部分 是 ?   這樣 要 怎麼 處理 呢   所以 就 有 機會   當日維 修好 囉   及 最 下面 三個   案件     也 沒 反應   ZF2   一個 小時 快 修 的 部分     恩恩     對   不是   如果 送修 的 話   有 可能 當天 就 好 嗎   請問 螢幕 觸控 按 了 沒 反應   沒 使用 單手 模式 下       任何 情況 那三個 按鍵 都 沒 反應   還有 螢幕 中間 部分 區塊 按 了 也 沒 反應   恩       好       我 了解 了   謝謝 你   550ML   現在 我 只能 轉換成 單手 操作 變成 小 螢幕 的 虛擬 三個 觸控 按鈕 才能 使用 那三個 案件 的 功能   是 因為 沒 辦法 按   我 才 轉換成 單手 模式 的'])
# randomForest_clf.predict_proba(['那一個 小時 預約 快 修   的 部分 是 ?   這樣 要 怎麼 處理 呢   所以 就 有 機會   當日維 修好 囉   及 最 下面 三個   案件     也 沒 反應   ZF2   一個 小時 快 修 的 部分     恩恩     對   不是   如果 送修 的 話   有 可能 當天 就 好 嗎   請問 螢幕 觸控 按 了 沒 反應   沒 使用 單手 模式 下       任何 情況 那三個 按鍵 都 沒 反應   還有 螢幕 中間 部分 區塊 按 了 也 沒 反應   恩       好       我 了解 了   謝謝 你   550ML   現在 我 只能 轉換成 單手 操作 變成 小 螢幕 的 虛擬 三個 觸控 按鈕 才能 使用 那三個 案件 的 功能   是 因為 沒 辦法 按   我 才 轉換成 單手 模式 的'])

In [13]:
from enum import Enum
class states(Enum):
    IDLE = 0
    RUNNING = 1

w2v_svr_pos_threshold = -0.2
w2v_svr_neg_threshold = -0.45

def score2text(score):
    if score > w2v_svr_pos_threshold:
        return '滿意'
    elif score < w2v_svr_neg_threshold:
        return '不滿意'
    else:
        return '中立'

In [14]:
def classifier(sentence_cut):
    # bag of word and classifier
    conbine_sentence = [' '.join(sentence_cut)]
    # LinearSVC
    if linearSVC_clf.predict(conbine_sentence)[0] == 1:
        print('LinearSVC: \t', '滿意\t')
    else:
        print('LinearSVC: \t', '不滿意')

    # Naive_Bayes
    if naive_bayes_clf.predict(conbine_sentence)[0] == 1:
        print('Naive_Bayes: \t', '滿意\t', naive_bayes_clf.predict_proba(conbine_sentence))
    else:
        print('Naive_Bayes: \t', '不滿意\t', naive_bayes_clf.predict_proba(conbine_sentence))
            
    # RandomForest
    if randomForest_clf.predict(conbine_sentence)[0] == 1:
        print('RandomForest: \t', '滿意\t', randomForest_clf.predict_proba(conbine_sentence))
    else:
        print('RandomForest: \t', '不滿意\t', randomForest_clf.predict_proba(conbine_sentence))
            
    # DecisionTree
    if decisionTree_clf.predict(conbine_sentence)[0] == 1:
        print('DecisionTree: \t', '滿意\t', decisionTree_clf.predict_proba(conbine_sentence))
    else:
        print('DecisionTree: \t', '不滿意\t', decisionTree_clf.predict_proba(conbine_sentence))

In [16]:
current_state = states.IDLE
window=3

sentence = ''
dialogue = []

print("請輸入'!start'以開始新對話，或者'!quit'結束程式")
while sentence != '!quit':
    sentence = input()
    if current_state == states.IDLE:
        if sentence == '!start':
            current_state = states.RUNNING
            print("新對話已開始，可輸入內容，或者'!end'以結束目前對話")
        elif sentence == '!quit':
            break
        else:
            print("請輸入'!start'以開始新對話，或者'!quit'結束程式")
    elif current_state == states.RUNNING:
        if sentence == '!end':
            print("此對話已結束，請輸入'!start'開始新對話，或者'!quit'結束程式")
            current_state = states.IDLE
            dialogue = []
        elif sentence.rstrip():
            dialogue.append(sentence)
            
            dialogue_in_window=''
            whole_dialogue = ''
            
            for d in dialogue[window*-1:]:
                dialogue_in_window += d + ' '
            
            for d in dialogue:
                whole_dialogue += d + ' '
            
            sentence_cut = jieba_cut(sentence)
            window_cut = jieba_cut(dialogue_in_window)
            dialogue_cut = jieba_cut(whole_dialogue)
            
            sentence_score = w2v_svr_score(sentence_cut)
            window_score = w2v_svr_score(window_cut)
            dialogue_score = w2v_svr_score(dialogue_cut)
            
            sen_text = score2text(sentence_score)
            win_text = score2text(window_score)
            dia_text = score2text(dialogue_score)
            
#             print(sentence_cut)
#             print(window_cut)
#             print(dialogue_cut)
            print('本句情緒分數:\t', sentence_score, '\t', sen_text)
            print('Window內情緒分數:\t', window_score, '\t', win_text)
            print('完整對話情緒分數:\t', dialogue_score, '\t', dia_text)
            print()
            print("本句滿意度: \t\t 不滿意機率 \t 滿意機率")
            classifier(sentence_cut)
            print()
            print("Window內滿意度: \t\t 不滿意機率 \t 滿意機率")
            classifier(window_cut)
            print()
            print("完整對話滿意度: \t\t 不滿意機率 \t 滿意機率")
            classifier(dialogue_cut)
            
#             print('Emotion degree: ', emotion_score_calculator(conbine_sentence))
            print('-------------------------------------------------------------')
    

請輸入'!start'以開始新對話，或者'!quit'結束程式
!start
新對話已開始，可輸入內容，或者'!end'以結束目前對話
您好我產品壞了
本句情緒分數:	 [-0.64070867] 	 不滿意
Window內情緒分數:	 [-0.64070867] 	 不滿意
完整對話情緒分數:	 [-0.64070867] 	 不滿意

本句滿意度: 		 不滿意機率 	 滿意機率
LinearSVC: 	 不滿意
Naive_Bayes: 	 滿意	 [[ 0.25524999  0.74475001]]
RandomForest: 	 不滿意	 [[ 0.75701912  0.24298088]]
DecisionTree: 	 不滿意	 [[ 0.6403865  0.3596135]]

Window內滿意度: 		 不滿意機率 	 滿意機率
LinearSVC: 	 不滿意
Naive_Bayes: 	 滿意	 [[ 0.25524999  0.74475001]]
RandomForest: 	 不滿意	 [[ 0.75701912  0.24298088]]
DecisionTree: 	 不滿意	 [[ 0.6403865  0.3596135]]

完整對話滿意度: 		 不滿意機率 	 滿意機率
LinearSVC: 	 不滿意
Naive_Bayes: 	 滿意	 [[ 0.25524999  0.74475001]]
RandomForest: 	 不滿意	 [[ 0.75701912  0.24298088]]
DecisionTree: 	 不滿意	 [[ 0.6403865  0.3596135]]
-------------------------------------------------------------
維修時間怎麼這麼久
本句情緒分數:	 [-1.29736682] 	 不滿意
Window內情緒分數:	 [-1.00551884] 	 不滿意
完整對話情緒分數:	 [-1.00551884] 	 不滿意

本句滿意度: 		 不滿意機率 	 滿意機率
LinearSVC: 	 不滿意
Naive_Bayes: 	 不滿意	 [[ 0.6212014  0.3787986]]
RandomForest: 	 不滿